[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/openlayer-ai/openlayer-python/blob/main/examples/tracing/openai-assistant/openai_assistant_tracing.ipynb)


# <a id="top">OpenAI assistant monitoring</a>

This notebook illustrates how to get started monitoring [OpenAI Assistants](https://platform.openai.com/docs/assistants/overview) with Openlayer.

In [ ]:
!pip install openlayer

## 1. Set the environment variables

In [ ]:
import os

import openai

# OpenAI env variables
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY_HERE"

# Openlayer env variables
os.environ["OPENLAYER_API_KEY"] = "YOUR_OPENLAYER_API_KEY_HERE"
os.environ["OPENLAYER_INFERENCE_PIPELINE_ID"] = "YOUR_OPENLAYER_INFERENCE_PIPELINE_ID_HERE"

## 2. Instantiate the OpenAI client

In [ ]:
openai_client = openai.OpenAI()

## 3. Create assistant, thread, and run it

In [ ]:
# Create the assistant
assistant = openai_client.beta.assistants.create(
    name="Data visualizer",
    description="You are great at creating and explaining beautiful data visualizations.",
    model="gpt-4",
    tools=[{"type": "code_interpreter"}],
)

In [ ]:
# Create a thread
thread = openai_client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "Create a data visualization of the american GDP.",
        }
    ]
)

In [ ]:
# Run assistant on thread
run = openai_client.beta.threads.runs.create(thread_id=thread.id, assistant_id=assistant.id)

In [ ]:
import time

from openlayer.lib import trace_openai_assistant_thread_run

# Keep polling the run results
while run.status != "completed":
    run = openai_client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

    # Trace the run with the Openlayer `trace_openai_assistant_thread_run`. If complete, the thread is sent to Openlayer
    trace_openai_assistant_thread_run(openai_client, run)

    time.sleep(5)